# HowTo -TreeGraph- From a curved skeleton to a tree graph

In this notebook you will learn:

1. how to connect to a ROMI database
2. how to transform a skeleton into a tree graph
3. how to visualize the tree graph

This notebook **assume** that you have:
- declared the `ROMI_DB` environment variable as the path to the database directory to use
- processed the test dataset with the _geometric pipeline,_ so we can access the fileset containing the data we want to start with...

Remember, the aim of this notebook is to show you how it works "under the hood".
This is not how you should process your data, that is done thanks to the `romi_run_task` CLI tool.

In [ ]:
import os

import ipywidgets as widgets
from plant3dvision.arabidopsis import compute_tree_graph
from plant3dvision.visu import plotly_skeleton, plotly_treegraph
from plantdb.fsdb import FSDB
from plantdb.io import read_json

# Connect to the database & get the initial data

If you did not declare a `ROMI_DB` environment variable, you can do it by uncommenting the next cell and setting it to the right value.

In [ ]:
# os.environ['ROMI_DB'] = "/path/to/test/data"

### Connect to the database

In [ ]:
db = FSDB(os.environ['ROMI_DB'])  # requires definition of this environment variable!
db.connect()

Once you are connected to the database, you can list the available scan *dataset* with `db.list_scans()`.

### Select a dataset

We now select a dataset (with the `Dropdown` widget) for the demo:

In [ ]:
scan_name = widgets.Dropdown(options=db.list_scans(), value=db.list_scans()[0], description='Dataset:')
display(scan_name)

In [ ]:
scan = db.get_scan(scan_name.value)

If you did not process this dataset yet, from the `plant3dvision` root directory, you can do it with:
```
romi_run_task AnglesAndInternodes $ROMI_DB/<selected_dataset> --config plant-3d-vision/configs/geom_pipe_real.toml
```

To list the available *filesets* in this *scan dataset*:

In [ ]:
scan.list_filesets()

### Get the skeleton fileset

The skeleton resulting from a _CurveSkeleton_ task is to be found in the 'CurveSkeleton*' fileset.

In [ ]:
skel_fs = scan.get_filesets(query={"task_name": "CurveSkeleton"})[0]
print(skel_fs.path().stem)

### Get and load the skeleton file

Once you have access to the right fileset, you may access the skeleton file and load it as follows:

In [ ]:
skel_file = skel_fs.get_files()[0]
skel = read_json(skel_file)

### Visualize the skeleton

It is possible to visualize the skeleton with the `plotly_skeleton` method from `plant3dvision.visu` as follows:

In [ ]:
fig = plotly_skeleton(skel)

In [ ]:
fig.show()

## Computing the tree graph

It is now time to process the *skeleton* to generate a *tree graph*.

This is done with the `compute_tree_graph` function from `plant3dvision.arabidopsis` as follows:

In [ ]:
tree = compute_tree_graph(skel["points"], skel["lines"], stem_axis=2, stem_axis_inverted=False)

In [ ]:
node_labels = ["_".join(tree.nodes[i].get('labels')) for i in range(len(tree.nodes))]
print(set(node_labels))

Note that the nodes of the tree graph have a "label" that describe their *nature* in:

* fruit
* stem
* stem_node

### Visualize the tree graph

Once the tree graph has been obtained, we can visualize it with the `plotly_treegraph` method from `plant3dvision.visu` as follows:

In [ ]:
fig = plotly_treegraph(tree)

In [ ]:
fig.show()

We may now **disconnect** from the database as we will not need it anymore:

In [ ]:
db.disconnect()